In [1]:
from keras.layers import Input,Dense,Lambda,concatenate
from keras.models import load_model,Model
import numpy as np
RL_mod = load_model('output/541/MOD/model.h5')
RL_tar = load_model('output/541/MOD/target_model.h5')
SL_mod = load_model('Read_out/Supervised_models/X1_mod_single_3cat.h5')

Using Theano backend.
/gpfs/hpchome/labash/miniconda3/envs/PT/lib/python3.5/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [2]:
RL_mod.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 633)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               63400     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 606       
_________________________________________________________________
lambda_1 (Lambda)            (None, 5)                 0         
Total params: 64,006
Trainable params: 64,006
Non-trainable params: 0
_________________________________________________________________


In [3]:
SL_mod.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 633)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               63400     
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 303       
Total params: 63,703
Trainable params: 63,703
Non-trainable params: 0
_________________________________________________________________


In [4]:
RL_tar.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 633)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               63400     
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 606       
_________________________________________________________________
lambda_2 (Lambda)            (None, 5)                 0         
Total params: 64,006
Trainable params: 64,006
Non-trainable params: 0
_________________________________________________________________


## Final model

In [30]:
def Merge_RL_SL(RL_mod,SL_mod):
    #Get the wanted layers.
    sll = SL_mod.get_layer(index=1)
    sll.name='SL_L'
    #Set supervized weights to be untrainable
    sll.trainable=False
    rll1 = RL_mod.get_layer(index=1)
    rll1.name='RL_L'
    rll2 = RL_mod.get_layer(index=2)
    rll2.name='RL_out'
    lambd=RL_mod.get_layer(index=3)
    lambd.name = 'RL_lamd_final'

    #Build the new model
    x = Input((633,))
    rl_way = rll1(x)
    sl_way = sll(x)
    sl_way = Dense(64,activation='tanh',name='SL_Dense64')(sl_way)
    sl_way = Dense(16,activation='tanh',name='SL_Dense16')(sl_way)
    merged = concatenate([rl_way,sl_way])
    output=Dense(6,name='output')(merged)
    out = lambd(output)
    fm = Model(inputs=[x],outputs=[out])
    #return fm
    #Update Dense layer weights
    w = fm.get_weights()
    w[8][:100,:] = RL_mod.get_weights()[2] #Kernel
    w[9] = RL_mod.get_weights()[3] #Bias
    fm.set_weights(w)
    return fm

In [31]:
fm = Merge_RL_SL(RL_mod,SL_mod)

In [32]:
fm.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 633)          0                                            
__________________________________________________________________________________________________
SL_L (Dense)                    (None, 100)          63400       input_6[0][0]                    
__________________________________________________________________________________________________
SL_Dense64 (Dense)              (None, 64)           6464        SL_L[6][0]                       
__________________________________________________________________________________________________
RL_L (Dense)                    (None, 100)          63400       input_6[0][0]                    
__________________________________________________________________________________________________
SL_Dense16

In [11]:
tar = Merge_RL_SL(RL_tar,SL_mod)

In [27]:
w = tar.get_weights()
w[4][100:] = fm.get_weights()[4][100:]
tar.set_weights(w)

In [10]:
fm.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 633)          0                                            
__________________________________________________________________________________________________
RL_L (Dense)                    (None, 100)          63400       input_3[0][0]                    
__________________________________________________________________________________________________
SL_L (Dense)                    (None, 100)          63400       input_3[0][0]                    
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 200)          0           RL_L[3][0]                       
                                                                 SL_L[3][0]                       
__________